### Index Question and there answers together

Answers which have questions should be index together - very successful !

Answer if it is associated with a question should be indexed with the question in elastic.

then reranked on the basis of answer text - ranking on the basis of answers alone yeilds bad results - worse than baseline

Reranking on the basis of whole Q+A


In [1]:
import glob
from tqdm.notebook import tqdm
import os
import jsonlines
import re
from elasticsearch import Elasticsearch
from collections import defaultdict
import pandas as pd

ans_path = '/data/szr207/dataset/ArqMath/jsons/answers/all.ans.jsonl'
ques_path = '/data/szr207/dataset/ArqMath/jsons/questions/all.ques.jsonl'

In [2]:
dict_ans = {}

with jsonlines.open(os.path.join(ans_path)) as reader:
        for obj in tqdm(reader):
            dict_ans[obj['post_id']] = obj

dict_ques = {}

with jsonlines.open(os.path.join(ques_path)) as reader:
        for obj in tqdm(reader):
            dict_ques[obj['post_id']] = obj

In [3]:
list_q_ans = []

for qid in tqdm(list(dict_ques.keys())):
    if dict_ques[qid]['accepted_answer_id'] in dict_ans:
        list_q_ans.append(dict_ques[qid]['accepted_answer_id'])
    if dict_ques[qid]['answers']:
        for ans in dict_ques[qid]['answers']:
            if ans in dict_ans:
                list_q_ans.append(ans)


In [4]:
len(list_q_ans)

1981788

In [5]:
list_q_ans = list(set(list_q_ans))
len(list_q_ans)

1435643

In [13]:
list_idx_qa = []

for a_id in tqdm(list_q_ans):
    ans_body = re.sub('<[^<]+?>', '',  dict_ans[a_id]['body'])
    qid = dict_ans[a_id]['parent_id']
    ques_body = re.sub('<[^<]+?>', '',  dict_ques[qid]['body'])
    ques_title = re.sub('<[^<]+?>', '',  dict_ques[qid]['title'])
    list_idx_qa.append({ 'post_id':a_id, 'body': ques_title + '. ' + ques_body + '. ' + ans_body })

In [8]:
list_idx_qa[:3]

[{5652: 'Floor function properties: [2x] = [x] + [ x + \\frac12 ] and [nx] = \\sum_{k = 0}^{n - 1} [ x + \\frac{k}{n} ] . I\'m doing some exercises on Apostol\'s calculus, on the floor function. Now, he doesn\'t give an explicit definition of [x], so I\'m going with this one:     DEFINITION Given  x\\in \\Bbb R, the integer part of x is the unique z\\in \\Bbb Z such that z\\leq x &lt; z+1 and we denote it by [x].   Now he asks to prove some basic things about it, such as: if n\\in \\Bbb Z, then [x+n]=[x]+n  So I proved it like this: Let z=[x+n] and z\'=[x]. Then we have that  z\\leq x+n&lt;z+1  z\'\\leq x&lt;z\'+1  Then z\'+n\\leq x+n&lt;z\'+n+1  But since z\' is an integer, so is z\'+n. Since z is unique, it must be that z\'+n=z.  However, this doesn\'t seem to get me anywhere to prove that  \\left[ {2x} \\right] = \\left[ x \\right] + \\left[ {x + \\frac{1}{2}} \\right]  in and in general that   \\left[ {nx} \\right] = \\sum\\limits_{k = 0}^{n - 1} {\\left[ {x + \\frac{k}{n}} \\right

In [10]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import os, uuid

es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)

if not es.ping():
    raise ValueError("Connection failed")

In [14]:
list_a = []
for obj in tqdm(list_idx_qa):
    dict_item = {
            "_index": "arq_ans_ques",
            "_id": uuid.uuid4(),
            "_source": obj }
    list_a.append(dict_item)

In [15]:
helpers.bulk(es, list_a)

(1435643, [])